In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import json
import folium
from glob import glob

from tqdm.notebook import tqdm

from sklearn import preprocessing 

# 커피지수 part_1

In [5]:
# 1. 데이터로부터 4개 커피숍 정보 추출
data_dir = './소상공인시장진흥공단_상가(상권)정보_서울_202112.csv'
df = pd.read_csv(f'{data_dir}/소상공인시장진흥공단_상가(상권)정보_서울_202112')

NotADirectoryError: ignored

In [7]:
df = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202112.csv', encoding = 'utf-8')

In [8]:
df

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,23324279,제중건강원,NaN,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,...,1156010600100410002034626,NaN,서울특별시 영등포구 영중로14길 11-17,150035.0,7250.0,NaN,1,NaN,126.907168,37.520613
1,24525909,민속악기사,NaN,D,소매,D04,취미/오락관련소매,D04A09,악기판매,G47593,...,1120012200101420011000227,NaN,서울특별시 성동구 용답5길 2,133849.0,4803.0,NaN,NaN,NaN,127.049018,37.566857
2,24715368,태평양진주,NaN,D,소매,D26,시계/귀금속소매,D26A01,시계/귀금속,G47830,...,1111015200100440000000001,금사랑투빌딩,서울특별시 종로구 서순라길 17-10,110390.0,3138.0,NaN,NaN,NaN,126.993530,37.571848
3,15554136,김선희꼼꼼국어교습소,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,학원-입시,P85501,...,1171011100101970003017195,NaN,서울특별시 송파구 위례성대로12길 31,138834.0,5640.0,NaN,2,NaN,127.121520,37.510967
4,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837.0,6572.0,NaN,1,NaN,126.991394,37.488375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220116,16439594,에프엔씨이새,NaN,D,소매,D06,가방/신발/액세서리,D06A05,액세서리판매,G47419,...,1174010900104550008000027,현대백화점,서울특별시 강동구 천호대로 1005,134022.0,5328.0,NaN,4,NaN,127.124428,37.538967
220117,14141515,목동제옥스,NaN,D,소매,D06,가방/신발/액세서리,D06A02,구두제화/캐주얼화,G47420,...,1147010200109160000021253,현대하이페리온,서울특별시 양천구 목동동로 257,158724.0,7998.0,NaN,1,NaN,126.874682,37.527147
220118,14104337,태양핸드백,NaN,D,소매,D06,가방/신발/액세서리,D06A10,가방/가죽제품소매,G47430,...,1138010700101110006024764,NaN,서울특별시 은평구 은평로6길 6,122907.0,3453.0,NaN,NaN,NaN,126.918918,37.599219
220119,14091596,중앙유니폼,NaN,D,소매,D05,의복의류,D05A14,유니폼/맞춤복/단체복,G47416,...,1162010200104870005015553,NaN,서울특별시 관악구 남부순환로 1535,151872.0,8762.0,NaN,3,NaN,126.921065,37.483329


In [9]:
# 업종분류코드: Q12 --> 카페zjvltiq
df = df[df.상권업종중분류코드 == 'Q12']

In [10]:
df = df[['상호명', '지점명', '시도명', '시군구명']]
print(df.shape)
df.head()

(12401, 4)


,상호명,지점명,시도명,시군구명
23,커피빈,코리아대학로대명거리점,서울특별시,종로구
99,요거프레소,쌍문점,서울특별시,도봉구
191,우성커피숍,NaN,서울특별시,양천구
198,버블베어,NaN,서울특별시,강서구
233,알뤼르,NaN,서울특별시,강남구


In [11]:
# 스타벅스 커피숍 case = False: 대소문자 무시하고 찾기
sb = df[df.상호명.str.contains('스타벅스|STARBUCKS', case=False)]
sb.head()

,상호명,지점명,시도명,시군구명
1544,스타벅스,동숭로아트점,서울특별시,종로구
2474,스타벅스남부터미널2점,남부터미널2점,서울특별시,서초구
2570,스타벅스,현대목동점,서울특별시,양천구
3589,스타벅스미아사거리역점,미아사거리역점,서울특별시,강북구
3608,스타벅스,가로수길점,서울특별시,강남구


In [17]:
cb = df[df.상호명.str.contains('커피빈|COFFEEBEAN', case=False, na=False)]
cb.head(3)

,상호명,지점명,시도명,시군구명
23,커피빈,코리아대학로대명거리점,서울특별시,종로구
390,커피빈코리아낙성대역점,코리아낙성대역점,서울특별시,관악구
1411,커피빈,코리아청담에스점,서울특별시,강남구


In [18]:
ed = df[df.상호명.str.contains('이디야|EDIYA', case=False, na=False)]
ed.head(3)

,상호명,지점명,시도명,시군구명
1131,이디야커피,신길역점,서울특별시,영등포구
1495,이디야커피,라이프점,서울특별시,영등포구
2397,이디야커피양재AT점,양재AT점,서울특별시,서초구


In [19]:
bd = df[df.상호명.str.contains('빽다방|PAIKSCOFFEE', case=False, na=False)]
bd.head(3)

,상호명,지점명,시도명,시군구명
7179,빽다방공덕새창로점,공덕새창로점,서울특별시,마포구
7789,빽다방서초우성점,서초우성점,서울특별시,서초구
18660,빽다방중계은행사거리점,중계은행사거리점,서울특별시,노원구


In [20]:
len(sb), len(cb), len(ed), len(bd)

(368, 126, 318, 88)

# 전국데이터에서 4개 매장 정보 추출

In [22]:
# 빈 데이터 프레임을 만들기
starbucks = pd.DataFrame(columns=[['상호명', '시도명', '시군구명']])
coffeebean = pd.DataFrame(columns=[['상호명', '시도명', '시군구명']])
ediya = pd.DataFrame(columns=[['상호명', '시도명', '시군구명']])
paik = pd.DataFrame(columns=[['상호명', '시도명', '시군구명']])

In [ ]:
# 디렉토리내 데이터파일을 for문으로 불러와 전처리
# for file in tqdm(glob(f'{data_dir}/*.csv')):
#      df = pd.read_csv(file)
#      df = df[df.상권업종중분류코드 == 'Q12']
#      df = df[['상호명', '시도명', '시군구명']]
#      sb = df[df.상호명.str.contains('스타벅스|STARBUCKS', case = False)]
#      cb = df[df.상호명.str.contains('커피빈|COFFEEBEAN', case=False, na=False)]
#      ed = df[df.상호명.str.contains('이디야|EDIYA', case=False, na=False)]
#      bd = df[df.상호명.str.contains('빽다방|PAIKSCOFFEE', case=False, na=False)]

#      starbucks = pd.concat([starbucks, sb])
#      coffeebean = pd.concat([coffeebean, cb])
#      ediya = pd.concat([ediya, ed])
#      paik = pd.concat([paik, bd])

In [ ]:
# starbucks.shape, coffeebean.shape, ediya.shape, paik.shape

In [26]:
sb.shape

(368, 4)

In [25]:
sb.to_csv('./스타벅스.csv')

In [31]:
cb.to_csv('./커피빈.csv')
ed.to_csv('./이디야.csv')

In [30]:
bd.to_csv('./빽다방.csv')